# Treinamento

## Inicialização

In [1]:
# Config Inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.metrics import *

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

### MLFlow

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

<Experiment: artifact_location='/app/mlruns/1', creation_time=1744227466344, experiment_id='1', last_update_time=1744227466344, lifecycle_stage='active', name='ecd15', tags={}>

In [3]:
seeds = [42, 47, 123]

## Dataset

O pré-processamento inicial foi realizado no dataset `brasile-real-state-dataset.csv`, onde as colunas de latitude e longitude foram convertidas em informações de cidade e estado. Esse processo resultou na criação do dataset `brasil_estado_cidade.csv`, que está sendo utilizado para o treinamento dos modelos apresentados a seguir.


### Leitura

In [5]:
# Descomentar, quando quiser carregar para o treinamento, dados originais defasados para primeira execução do treinamento
#shutil.copyfile("dataset/brasil_estado_cidade.csv", "dataset/dados_treinamento.csv")


# Carregar o conjunto de dados
dados = pd.read_csv("dataset/dados_treinamento.csv", encoding="utf-8")

# Eliminando registros com valores null
dados.dropna(inplace=True)

# float64
dados = dados.astype({col: 'float64' for col in dados.select_dtypes(include='int').columns})

def remover_outliers_por_cidade(df):
    # Remove outliers da coluna 'price_brl' agrupando por cidade.

    def remover_outliers_grupo(grupo):
        # Remove outliers de um grupo usando o método Z-score.
        z_scores = np.abs(stats.zscore(grupo["price_brl"]))
        return grupo[(z_scores < 3)]

    df_filtrado = df.groupby("city").apply(remover_outliers_grupo).reset_index(drop=True)
    return df_filtrado

dados = remover_outliers_por_cidade(dados)

dados.sample(10)

/tmp/ipykernel_1394/3584315994.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby("city").apply(remover_outliers_grupo).reset_index(drop=True)


,id,property_type,state,region,lat,lon,area_m2,price_brl,city
8160,6429.0,house,Rio de Janeiro,Southeast,-22.916160,-43.355623,140.0,454317.88,Rio de Janeiro
1416,4024.0,house,Rio de Janeiro,Southeast,-22.862112,-42.021259,230.0,768558.29,Cabo Frio
4908,2230.0,apartment,Rio Grande do Sul,South,-30.032711,-51.211208,109.0,393742.15,Porto Alegre
7566,5544.0,apartment,Rio de Janeiro,Southeast,-22.944958,-43.240059,148.0,908635.80,Rio de Janeiro
5374,2956.0,house,Rio Grande do Sul,South,-30.150503,-51.213573,110.0,548970.21,Porto Alegre
4806,2081.0,apartment,Rio Grande do Sul,South,-30.027777,-51.162804,99.0,648782.97,Porto Alegre
3291,4096.0,apartment,Rio de Janeiro,Southeast,-22.906288,-43.099586,62.0,373550.26,Niterói
5368,2946.0,house,Rio Grande do Sul,South,-30.117683,-51.260231,186.0,1396270.36,Porto Alegre
5533,3217.0,apartment,Rio Grande do Sul,South,-30.031361,-51.157860,125.0,1009595.32,Porto Alegre
7261,5109.0,apartment,Rio de Janeiro,Southeast,-22.956388,-43.375317,174.0,1261994.16,Rio de Janeiro


### Tratamento Features

In [5]:
# Exemplo de pré-processamento
x_features = dados.drop(["price_brl"], axis=1)  # Features
y_target = dados["price_brl"]  # Variável alvo

# Identificando colunas numéricas e categóricas
numeric_features = x_features.select_dtypes(include=['number']).columns
categorical_features = x_features.select_dtypes(include=['object']).columns

# Criando transformadores
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Criando o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### Divisão dos Dados

In [6]:
test_sizes = [0.1, 0.2, 0.3]
#x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.2, random_state=42)

## Modelos

- Árvore de Decisão: `DecisionTreeRegressor`
- Regressão Linear: `LinearRegression`
- Random Forest: `RandomForestRegressor`
- Gradient Boosting: `GradientBoostingRegressor`
- XGBoost: `XGBRegressor`

In [7]:
models = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

## Execução

In [8]:
# para cada semente 
for seed in seeds:
    # para cada tamanho de teste
    for test_size in test_sizes:
        # Dividir os dados em conjuntos de treino e teste
        x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=test_size, random_state=seed)

        # Para cada modelo
        for model_name, model in models.items():
            # Criar o pipeline
            pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('model', model)])

            # Treinar o modelo
            pipeline.fit(x_train, y_train)

            # Fazer previsões
            y_pred = pipeline.predict(x_test)

            # Avaliar o modelo
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            
            print(f"Modelo: {model_name}, Test Size: {test_size}, Seed: {seed}")
            print(f"MSE: {mse}, R2: {r2}, MAE: {mae}")
            
            # Logar os resultados no MLflow
            with mlflow.start_run():
                mlflow.log_param("model_type", model_name)
                mlflow.log_param("test_size", test_size)
                mlflow.log_param("random_seed", seed)
                
                # registrar métricas
                mlflow.log_metric("mse", mse)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)

                # Logar o modelo
                signature = infer_signature(x_test, y_pred)
                
                registered_model_name= model_name.lower().replace(" ", "_") + "_model"
                 
                mlflow.sklearn.log_model(
                    sk_model=pipeline,
                    artifact_path=registered_model_name,
                    signature=signature,
                    registered_model_name= registered_model_name,
                )
                
            mlflow.end_run()
                

Modelo: Decision Tree, Test Size: 0.1, Seed: 42
MSE: 774057741995.8984, R2: 0.7670971810981704, MAE: 349363.0108622449


Registered model 'decision_tree_model' already exists. Creating a new version of this model...
Created version '26' of model 'decision_tree_model'.


Modelo: Linear Regression, Test Size: 0.1, Seed: 42
MSE: 2199784252261.418, R2: 0.33811662162762457, MAE: 892908.60254239


Registered model 'linear_regression_model' already exists. Creating a new version of this model...
Created version '26' of model 'linear_regression_model'.


Modelo: Random Forest, Test Size: 0.1, Seed: 42
MSE: 499186417304.9078, R2: 0.8498020012718455, MAE: 323265.798247449


...

Registered model 'random_forest_model' already exists. Creating a new version of this model...
Created version '33' of model 'random_forest_model'.


Modelo: Gradient Boosting, Test Size: 0.3, Seed: 123
MSE: 922498727862.6497, R2: 0.7571880920801617, MAE: 493347.57351917034


Registered model 'gradient_boosting_model' already exists. Creating a new version of this model...
Created version '33' of model 'gradient_boosting_model'.


Modelo: XGBoost, Test Size: 0.3, Seed: 123
MSE: 597322441521.3182, R2: 0.8427781011631683, MAE: 390276.3078306229


Registered model 'xgboost_model' already exists. Creating a new version of this model...
Created version '33' of model 'xgboost_model'.
